In [1]:
"""
Created on Wed Oct 19 15:48:00 2018

@author: Daniel Cuesta, Alejandro Garo
"""

import string, random
import numpy as np
import matplotlib.pyplot as plt
import sys
import time

import queue

from sklearn.linear_model import LinearRegression

import networkx as nx

In [4]:
m_mg = np.random.binomial(3,0.5,(3,3))
print(m_mg)

[[2 0 1]
 [2 1 2]
 [2 2 1]]


In [5]:
# Ejercicio 1B: Agrandando nuestra ED
def m_mg_2_d_mg(m_mg):
    """
    De un multigrafo dirigido sin pesos, genera un diccionario de adyacencia a partir de su matriz de adyacencia.
    g[u]: diccionario de diccionarios con los nodos destino y el num de caminos con su peso (1 por ser unweighted)
    g[u][v]: num de caminos con su peso (1)
    Devuelve la matriz de adyacencia.

    Parámetros
    ----------
    m_mg: matriz de adyacencia, contiene el numero de caminos del nodo u al uv
    """
    d_mg = {}
    
    for i in range(m_mg.shape[0]):
        d_mg[i] = {}
        for j in range(m_mg.shape[1]):
            d_mg[i][j] = {}
            for n_edge in range(m_mg[i][j]):
                d_mg[i][j].update({n_edge : 1})
    return d_mg

def rand_unweight_multigraph(n_nodes, num_max_multiple_edges = 3, prob = 0.5):
    """
    Genera un multigrafo dirigido sin pesos haciendo uso del metodo binomial del paquete random de numpy.
    Parámetros
    ----------
    n_nodes: numero de nodos
    num_max_multiple_edges: numero maximo de aristas de un nodo
    prob: probabilidad de generacion de las aristas
    """
    return np.random.binomial(num_max_multiple_edges, prob, (n_nodes,n_nodes))


def graph_2_multigraph(d_g):
    """
    Convierte un grafo que usa nuestra ED diccionario anterior a la nueva

    Parámetros
    ----------
    d_g: grafo usando la ED diccionario anterior
    """
    d_mg = {}
    for u in d_g.keys():
        d_mg.update({u : {}})
        # print(d_mg[u])
        #
        #
        #TODO saber si hay que meter caminos repetidos
        for v in d_g[u].keys():
            d_mg[u].update({v:{0:1}})
        print(d_mg[u])
        # if d_mg[u][v] is v:
        #     d_mg[u].update({int(d_mg[u][v]) + 1 : 1})
        #     print(d_mg[u])
                
            
def print_multi_graph(d_mg):
    """
    Imprime el multigrafo dirigido sin pesos
    
    Parámetros
    ----------
    d_mg: multigrafo usando la ED diccionario
    """
    for u in d_mg.keys():
        # print(d_mg[u])
        for v in d_mg[u].keys():
            l_w = []
            for w in d_mg[u][v].keys():
                l_w.append([d_mg[u][v][w]])
            print("(" + str(u) + "," + str(v) + ")" + ": " + str(l_w))

In [6]:
# Prueba

# m_mg a d_mg
# m_mg = rand_unweight_multigraph(3, num_max_multiple_edges = 3, prob = 0.5)
# d_mg = m_mg_2_d_mg(m_mg)
# print(m_mg)
# print_multi_graph(d_mg)

# d_g a d_mg
d_g = {
    0: {0: 1, 1: 1, 2: 1},
    1: {0:1, 2: 1},
    2: {3: 1},
    3: {1: 1}
}
graph_2_multigraph(d_g)

{0: {0: 1}, 1: {0: 1}, 2: {0: 1}}
{0: {0: 1}, 2: {0: 1}}
{3: {0: 1}}
{1: {0: 1}}


In [7]:
# Pruebas con NX MultiDiGraph
mg = nx.MultiDiGraph()

l_e = [(0,1,10), (0,2,1), (1,2,1), (2,3,1), (3,1,1)]

#mg.add_weighted_edges_from(l_e)
mg.add_weighted_edges_from(l_e)


print(mg[0])

print(mg[0][1])

{1: {0: {'weight': 10}}, 2: {0: {'weight': 1}}}
{0: {'weight': 10}}


In [69]:
# Ejercicio 2A: Detectando caminos eulerianos.

def adj_inc_directed_multigraph(d_mg):
    """
    Devuelve las adyacencias e incidencias de cada vertice del multigrafo d_mg
    
    Parámetros
    ----------
    d_mg: multigrafo usando la ED diccionario
    """
    
    inc = [0]*len(d_mg)
    adj = [0]*len(d_mg)
    for u in d_mg.keys():
        for v in d_mg[u].keys():
            #evitamos contar los ciclos en un mismo nodo
            if u != v:
                adj[u] = adj[u] + len(d_mg[u][v])
                inc[u] = inc[u] + len(d_mg[v][u])

    return (inc, adj)


In [84]:
m_mg = rand_unweight_multigraph(3, num_max_multiple_edges = 3, prob = 0.5)
d_mg = m_mg_2_d_mg(m_mg)
print(m_mg)
print_multi_graph(d_mg)
inc, adj = adj_inc_directed_multigraph(d_mg)
print(inc, adj)


[[1 0 1]
 [1 1 1]
 [1 0 1]]
(0,0): [[1]]
(0,1): []
(0,2): [[1]]
(1,0): [[1]]
(1,1): [[1]]
(1,2): [[1]]
(2,0): [[1]]
(2,1): []
(2,2): [[1]]
[2, 0, 2] [1, 2, 1]


In [89]:
def isthere_euler_path_directed_multigraph(d_mg):
    """
    Devuelve True o False segun haya o no un camino euleriano en el multigrafo d_mg
    
    Parámetros
    ----------
    d_mg: multigrafo usando la ED diccionario
    """
    
    #inc, adj = adj_inc_directed_multigraph(d_mg)
    
    inc = [2, 1, 2]
    adj = [1, 2, 1]
    if sum(inc) != sum(adj):
        print("polla")
    for u in range(len(inc)):
        if inc[u]>adj[u]+1 or inc[u]+1<adj[u] or (inc[u] == 0 and adj[u] == 0):
            print("Este grafo e una mierda")
            return False
        else:
            print("HOLA")
    return True


In [90]:
isthere_euler_path_directed_multigraph(d_mg)

polla
HOLA
HOLA
HOLA


True

In [ ]:
def first_last_euler_path_directed_multigraph(d_mg):
    """
    Devuelve  del multigrafo d_mg
    
    Parámetros
    ----------
    d_mg: multigrafo usando la ED diccionario
    """
    
    